# load libraries and dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
data=pd.read_csv('data/train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data=np.array(data)
m,n=data.shape()
np.random.shuffle(data)

# converting data into numpy arrays as it is the data structure best for neural network rather than pandas dataframe

# row and column representation of values

In [ ]:
# 1000 data points are set aside for validation
# remaining become features
data_valid=data[0:1000].T
y_valid=data_dev[0]
x_valid=data_dev[1:n]

# training set is remanining data which are transposed
data_train=data[1000:m].T
y_train=data_train[0]
x_train=data_dev[1:n]

# 1000 data are set aside for validation purpose as it is important to prevent overfitting so the model can generalized unseen data

# matrix rows need to be transposed for row to columns vice versa

# the rest data is set aside for testing